In [3]:
import re
import csv
from math import log, exp   # 기본 자연로그
from collections import Counter

In [2]:
def open_csv():
    f = open('./data/IMDB Dataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)
    
    pos_doc = []
    neg_doc = []
    
    next(csvreader) # header 제외
    for line in csvreader:
        if line[1] == 'positive':
            pos_doc.append(line[0]) # 그냥 line으로 쓰면 list로 들어가게 됨
        else:
            neg_doc.append(line[0])
    
    train_datas = [[], []]
    train_datas[0] = neg_doc
    train_datas[1] = pos_doc
    
    # train data의 0번째 인덱스와 1번째 인덱스 문자열
    # 리스트는 토큰화가 어려워서 전부 조인을 통해서 하나의 문자열로 바꾼다.
    return [' '.join(train_datas[0]), ' '.join(train_datas[1])]

In [3]:
# train_data가 긍정이면, P(test_data|긍정)
# train_data가 부정이면, P(test_data|부정)
# 만약 train_data에 테스트할 단어가 없으면, 확률을 알 수 없음 -> nowords_weight(하이퍼파라미터)
def calculate_doc_prob(train_data, test_data, nowords_weight):
    log_prob = 0 # 확률을 곱하는 것은 log 확률을 더하는 것과 같다
                 # 확률이 0에 수렴하지 않게 log 확률을 사용
        
    # train------------------------------------------------------
    # stopwords 제거
    sw_train_data = re.compile("[^\w]").sub(' ', train_data.lower())
    # 토큰화
    sw_train_token = sw_train_data.split() # 빠르게 단어 별로 tokenize
    # Bow
    train_vector = dict(Counter(sw_train_token)) # Counter를 쓰면 dict처럼 보이지만 Counter 객체로 나옴
    
    # test-------------------------------------------------------
    # stopwords 제거
    sw_test_data = re.compile("[^\w]").sub(' ', test_data.lower())
    # 토큰화
    sw_test_token = sw_test_data.split()
    # Bow
    test_vector = dict(Counter(sw_test_token)) 
    
    # 전체 단어의 갯수
    total_wc = len(sw_train_token)
    
    # log(P(test|긍정))
    # test 문장에 있는 단어 별로 긍정인 문장에서 등장한 확률을 다 곱해준다.
    # <=> log 확률을 다 더해준다
    for word in test_vector:
        if word in train_vector:
            # P(단어|긍정)
            log_prob += log(train_vector[word]/total_wc)
        else:
            log_prob += log(nowords_weight/total_wc) # nowords_weight는 주로 (0, 1) 값을 대입 
                                                     # 1번 등장한 단어보다 비중을 줄여주기 위함
                                                     # 강사님은 0.1 사용
    return log_prob

In [4]:
def naive_bayes(train_datas, test_data, pos_prob, neg_prob):
    # P(긍정|test) = P(test|긍정) * P(긍정) / P(test)
    # P(test)로 나누는 부분은 중복되는 부분이므로 생략 
    # 정확한 확률을 구하는 것이 목적이 아님
    test_pos_prob = calculate_doc_prob(train_datas[1], test_data, 0.1) + log(pos_prob) 
    # P(부정|test) = P(test|부정) * P(부정) / P(test)
    test_neg_prob = calculate_doc_prob(train_datas[0], test_data, 0.1) + log(neg_prob)
    
    # ------------정규화 작업
    # 현재 구한 test_pos_prob, test_neg_prob은 절대값이 큰 음수라서 보기 쉽게 변환
    # 긍정, 부정의 상대적 확률
    # 긍정을 1로 놨을 때 부정의 확률 or 부정을 1로 놨을 때 긍정의 확률 
    maxprob = max(test_neg_prob, test_pos_prob)
    test_pos_prob -= maxprob
    test_neg_prob -= maxprob
    test_pos_prob = exp(test_pos_prob)
    test_neg_prob = exp(test_neg_prob)
    normalized_prob = [test_neg_prob/(test_neg_prob+test_pos_prob), test_pos_prob/(test_neg_prob+test_pos_prob)]
    
    return normalized_prob

In [10]:
def start():
    train_datas = open_csv() 
    
    # 긍정 확률을 추정하고자 하는 새로운 문장
    test_data = input()
    # 위 두 데이터를 이용해서 확률을 계산
    prob = naive_bayes(train_datas, test_data, 0.5, 0.5)
    
    print(f"리뷰가 부정적인 확률 : {round(prob[0], 5)}, 긍정적일 확률 : {round(prob[1], 5)}")

In [15]:
start()

 Original Score: 2.5/4 Despite all indicators pointing to romantic comedy, the film is devoid of romance. Slate leads an exceptional cast and there are funny moments but the film gets lost in translation from the novel.
리뷰가 부정적인 확률 : 0.7928, 긍정적일 확률 : 0.2072


---

# train, test를 분리해서 모델 평가하는 부분을 추가

In [12]:
def start_validate():
    train_datas, test_data = open_csv_validate()
    
    prob_neg = naive_bayes(train_datas, test_data[0], 0.5, 0.5)
    prob_pos = naive_bayes(train_datas, test_data[1], 0.5, 0.5)
    
    print(f"부정 텍스트가 부정적일 확률 : {round(prob_neg[0], 5)}, 긍정적인 확률 : {round(prob_neg[1], 5)}")
    print(f"긍정 텍스트가 부정적일 확률 : {round(prob_pos[0], 5)}, 긍정적인 확률 : {round(prob_pos[1], 5)}")
    
    
    
def open_csv_validate():
    f = open('./data/IMDB Dataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)
    
    pos_doc = []
    neg_doc = []
    
    next(csvreader) 
    for line in csvreader:
        if line[1] == 'positive':
            pos_doc.append(line[0]) 
        else:
            neg_doc.append(line[0])
    
    # 각 문서가 25000개 정도씩 있음(4:1로 분리)
    train_datas = [[], []]
    train_datas[0] = neg_doc[:20000]
    train_datas[1] = pos_doc[:20000]
    
    test_datas = [neg_doc[20000:], pos_doc[20000:]]
    
    
    return [' '.join(train_datas[0]), ' '.join(train_datas[1])], [' '.join(test_datas[0]), ' '.join(test_datas[1])]




def naive_bayes(train_datas, test_data, pos_prob, neg_prob):
    test_pos_prob = calculate_doc_prob(train_datas[1], test_data, 0.1) + log(pos_prob) 
    test_neg_prob = calculate_doc_prob(train_datas[0], test_data, 0.1) + log(neg_prob)
     
    maxprob = max(test_neg_prob, test_pos_prob)
    test_pos_prob -= maxprob
    test_neg_prob -= maxprob
    test_pos_prob = exp(test_pos_prob)
    test_neg_prob = exp(test_neg_prob)
    normalized_prob = [test_neg_prob/(test_neg_prob+test_pos_prob), test_pos_prob/(test_neg_prob+test_pos_prob)]
    
    return normalized_prob



def calculate_doc_prob(train_data, test_data, nowords_weight):
    log_prob = 0 
        
   
    sw_train_data = re.compile("[^\w]").sub(' ', train_data.lower())

    sw_train_token = sw_train_data.split() 
    train_vector = dict(Counter(sw_train_token))
    

    sw_test_data = re.compile("[^\w]").sub(' ', test_data.lower())
    sw_test_token = sw_test_data.split()
    test_vector = dict(Counter(sw_test_token)) 
    
   
    total_wc = len(sw_train_token)
    
    for word in test_vector:
        if word in train_vector:
            log_prob += log(train_vector[word]/total_wc)
        else:
            log_prob += log(nowords_weight/total_wc) 
            
    return log_prob

start_validate()

부정 텍스트가 부정적일 확률 : 1.0, 긍정적인 확률 : 0.0
긍정 텍스트가 부정적일 확률 : 0.0, 긍정적인 확률 : 1.0
